# Initial setup

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

Import all needed packages

In [ ]:
import os
import cv2
import tensorflow as tf
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import json
from six.moves import urllib
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [ ]:
from keras import applications
preprocess_input = applications.mobilenet_v2.preprocess_input 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
tf.config.experimental.set_memory_growth(gpus[1], True)

In [ ]:
#! unzip ~/Documents/Dataset/binary/trainBN600.zip -d ~/Documents/Dataset/binary

## Set useful paths

The folder structure is the following: there is a main folder *Dataset* with inside *binary* folder. All elements for the training are in the latter, in particular in two folders *Persona* and *Others* in *path_train*.

The class we want to recognize among all is therefore the *target class Persona*. 

In [ ]:
path_ds = "Dataset"
path_binary = os.path.join(path_ds, "binary")
path_train = os.path.join(path_binary, "trainBN")
if not os.path.exists(path_train):
      os.makedirs(path_train)
target = "Persona"

In [ ]:
path_train_Persona = os.path.join(path_train, "Persona")
if not os.path.exists(path_train_Persona):
      os.makedirs(path_train_Persona)
path_train_Others = os.path.join(path_train, "Others")  
if not os.path.exists(path_train_Others):
      os.makedirs(path_train_Others)

We print how many people images and how many alien images are present

In [ ]:
print("There are ", len(os.listdir(path_train_Persona)), " images from Target dataset")

In [ ]:
print("There are ", len(os.listdir(path_train_Others)), " pre-processed images from class Others")

# Training part

## Build the model


We instantiate a unique MobileNetV2 in *base_model*. MobileNetV2 by Google belongs to MobileNets family, efficient and optimized architectures for mobile devices. It is fast and provides high accuracy, requiring few parameters and low computational power, also compared to previous versions.

• We load *weights* pre-trained on ImageNet, without including the default top part with 1000 neurons;

• The *input shape* of images is set to (224, 224, 3);

• The hyperparameter alpha, belonging to range (0, 1] and known as the width multiplier that determines the number of filters at each layer, is set to its default value 1;

• A *global average pooling layer* is inserted after the the last convolutional block, passing from a 4D output tensor of shape (batch_size, 7, 7, 1280) to a flattened 2D output tensor of shape (batch_size, 1280).

In [ ]:
alpha = 1.0 #for MobileNetV2
base_model = applications.MobileNetV2(include_top=False, 
                                      input_shape=(224, 224, 3), 
                                      alpha=alpha, 
                                      weights='imagenet',
                                      pooling="avg")

In [ ]:
base_model.summary()

• A *fully connected layer* with one neuron and therefore a sigmoid activation function is attached, in order to compute binary predictions

In [ ]:
predictions = Dense(1, activation='sigmoid')(base_model.output)

The keras model is defined, having:

• 1 input, the base model input that is an image batch of size (batch_size, 224, 224, 3);

• 1 output, the binary classification outcome. 

In [ ]:
model = keras.Model(inputs=base_model.input, outputs = predictions)

We print the output shape

In [ ]:
print(model.output) #shape=(None, 1)

We visualize properties of all layers that are part of the model and their number

In [ ]:
model.summary()

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
print("Number of layers in the model: ", len(model.layers))

In the training phase some layers of the network are *frozen*, to preserve
imported parameters pre-trained on ImageNet. This means we take low-level
features learned in a different classification task, by leveraging them
in our problem.

In MobileNetV2 we choose to initially freeze all blocks until block 13, having 40 unfrozen layers over the whole 157 layers.

In [ ]:
#Fixed weights
for layer in model.layers:
    if layer.name == "block_13_expand": # "block5_conv1": for VGG16
        break
    else:
        layer.trainable = False

In [ ]:
#model.trainable = True

In [ ]:
k=0
for layer in model.layers:
    #print(layer, layer.trainable)
    if layer.trainable == True:
        k=k+1
print("Layers with trainable=True: ", k, "")

In [ ]:
for layer in model.layers:
    print(layer, layer.trainable)

## Compile the model

The model is compiled defining:

• the *batch_size* of the input batch composed by images from the two classes;

• the *optimizer* as the gradient descent algorithm, employed with a very
low learning rate lr = 0.00005 and a weight decay of 0.00005;

• the loss as the *binary crossentropy*, consistent with network output;

• the monitored metric as the *accuracy*.

In [ ]:
#batch_size = 256 
batch_size = 32 
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.00005, decay=0.00005), #, momentum=0.9),
    loss='binary_crossentropy',
    metrics=["accuracy"],
)

## Create th ImageDataGenerator object for training

In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
input_imgen = ImageDataGenerator(preprocessing_function = preprocess_input)

gen1 = input_imgen.flow_from_directory(path_train,
                                        target_size = (224 ,224),
                                        class_mode = 'binary',
                                        batch_size = batch_size,
                                        shuffle=True)

In [ ]:
print(gen1.class_indices)   #{'Others': 0, 'Persona': 1}

Epochs are delineated from the size of the target dataset.

The number of epochs is set to 400, taking care to save intermediate models every 50 epochs to properly study the evolution of tested metrics.

In [ ]:
train_size = len(os.listdir(path_train_Persona)) #6000
epochs = 400

In [ ]:
print(train_size)

## Train with *fit*

In [ ]:
history = model.fit(gen1,
                    epochs = epochs,
                    steps_per_epoch = train_size // batch_size,
                    #use_multiprocessing=True,
                    #shuffle=False
                    )

In [ ]:
model.metrics_names

Retrive losses and accuracy from history

In [ ]:
# Retrieve losses and accuracy
final_loss = history.history['loss']
acc = history.history['dense_accuracy']

# Get number of epochs
epochs = range(len(total_loss))

print("Loss = ", final_loss)
print("Accuracy (dense) = ", acc)

## Train with *train_on_batch* (suggested)

Here, values of losses are stored every 10 batch iterations, in order to understand what happens during each epoch.

In [ ]:
final_loss = final_loss
acc = acc

In [ ]:
final_loss= []
acc= []

In [ ]:
n_batches = train_size // batch_size
print("Number of batches : ", n_batches)

for epoch in range(epochs):
    print("\nEpoch ", epoch+1 , "/", epochs)
  
    for i in range(n_batches):
        print("Processing batch...  ", i)
        batch = next(gen1)
        #print(type(batch), batch[0].shape, batch[1].shape)
        loss, accuracy = model.train_on_batch(batch[0], batch[1])
        #Print the total loss every 10 iterations
        if i % 10 == 0:
            print("\nLoss after iteration ", i, " is ", loss)
            final_loss.append(loss)
            acc.append(accuracy)
    if (epoch+1) % 50 == 0:
        my_model = "my_model_binary4000_"+ str(epoch+1) +".h5"
        path_model = os.path.join(path_ds, my_model)  #/content/drive/My Drive/my_model.h5
        model.save(path_model)

    print("Loss at the end of epoch " , epoch+1, ": ", loss)

## Retrieve losses stored in folder *Dataset* if needed

In [ ]:
path_loss = os.path.join(path_ds, "loss19.json") 
path_lc = os.path.join(path_ds, "l_c19.json")
path_ld = os.path.join(path_ds, "l_d19.json")

In [ ]:
with open(path_loss, 'r') as fp:
    total_loss = json.load(fp)
with open(path_lc, 'r') as fp:
    l_c = json.load(fp)
with open(path_ld, 'r') as fp:
    l_d = json.load(fp)

## Plot loss

In [ ]:
print(np.min(final_loss))
print(np.max(acc))

Plot loss and accuracy

In [ ]:
import matplotlib.pyplot as plt
plt.plot(final_loss, label="Loss")
plt.plot(acc, label="Accuracy")
plt.xlabel("training steps")
plt.legend()
plt.show()

Plot loss

In [ ]:
plt.plot(final_loss, label="Loss")
plt.xlabel("training steps")
plt.legend()
#plt.xscale('log')
plt.show()

Plot accuracy

In [ ]:
plt.plot(acc, label="Accuracy")
plt.xlabel("training steps")
#plt.xscale('log')
plt.legend()
plt.show()

## Save losses on folder *Dataset* if needed


In [ ]:
path_loss = os.path.join(path_ds, "loss20.json") #/content/drive/My Drive/loss.json
path_lc = os.path.join(path_ds, "l_c20.json")
path_ld = os.path.join(path_ds, "l_d20.json")

In [ ]:
with open(path_loss, 'w') as fp:
    json.dump(total_loss, fp)
with open(path_lc, 'w') as fp:
    json.dump(l_c, fp)
with open(path_ld, 'w') as fp:
    json.dump(l_d, fp)

## Save trained model

In [ ]:
path_model = os.path.join(path_ds, "my_modelbin_200.h5")  #/content/drive/My Drive/my_model.h5
model.save(path_model)  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model